# Binning diagnostic

In [105]:
from simpl.utils import load_datafile, prepare_data
from simpl.diagnostics import binning_diagnostic

raw = load_datafile("gridcelldata.npz")
data = prepare_data(Y=raw["Y"][:, :50], Xb=raw["Xb"], time=raw["time"])

binning_diagnostic(data)

Fraction of bins with >= 2 active neurons: 94.7%
Average spatial information per bin:        1.1 bits per bin (10.6 bits/s)


(0.9474166666666667, 1.0553795099258423)